# from bold to neural state

In [2]:
from scipy.stats import gamma
import numpy as np
from math import ceil

def gammaHRF(TR, paras=None, len_of_seconds=32, onset_seconds=0):
    
    if paras is None:
        paras = [6,16,1,1,6]

    dt = TR/16
    u = np.array(range(0, ceil(len_of_seconds/dt))) - ceil(onset_seconds/dt)
    
    tmp1 = gamma.pdf(u, a = paras[0]/paras[2], scale = 1/(dt/paras[2])) 
    tmp2 = gamma.pdf(u, a = paras[1]/paras[3], scale = 1/(dt/paras[3]))/paras[4]       
    hrf = tmp1 - tmp2

    hrf = hrf[np.array(range(0, ceil(len_of_seconds/TR)))*16 + 1]

    hrf = hrf/sum(hrf)

    return hrf

In [114]:
def gammaHRF_d(TR, paras=None, len_of_seconds=32, onset_seconds=0):
    ## derivative of gamma HRF
    
    if paras is None:
        paras = [6,16,1,1,6]

    dt = TR/16

    u = np.array(range(0, ceil(len_of_seconds/dt))) - ceil(onset_seconds/dt)

    def dgamma_d(x,a,r):
        return (a-1)*gamma.pdf(x, a-1, scale = 1/r)-r*gamma.pdf(x,a,scale = 1/r)

    tmp1 = dgamma_d(u, paras[0]/paras[2], dt/paras[2])
    tmp2 = dgamma_d(u, paras[1]/paras[3], dt/paras[3])/paras[4]
    hrf = tmp1 - tmp2
    idx = np.array(range(0, ceil(len_of_seconds/TR)))*16 + 1

    hrf = hrf[idx]

    hrf = hrf/sum(hrf)

    return hrf

gammaHRF_d(2.2).round(2)

array([ 0.  ,  0.54,  0.84,  0.43,  0.08, -0.13, -0.23, -0.22, -0.16,
       -0.09, -0.04, -0.02, -0.01, -0.  , -0.  ])

In [3]:
from numpy.fft import fft, ifft, ifftshift

def wiener_deconvolution(signal, kernel, lambd):
    
    kernel = np.hstack((kernel, np.zeros(len(signal) - len(kernel)))) # zero pad the kernel to same length
    H = fft(kernel)
    deconvolved = np.real(ifft(fft(signal)*np.conj(H)/(H*np.conj(H) + lambd**2)))
    return deconvolved

$$\ G(f) = \frac{H^*(f)S(f)}{ |H(f)|^2 S(f) + N(f) }$$

lambd-estimated noise lev
lambd should certainly be described more precisely as the inverse of SNR(Signal-to-noise ratio)

In [4]:
import pandas as pd
import numpy as np

filename = "NCANDA_S01010_run-01.feat__filtered_func.nii.gz.1D"
df = pd.read_csv(filename, sep='\t', header=None)

In [5]:
df.head(10)

,0,1,2,3,4,5,6,7,8,9,...,519,520,521,522,523,524,525,526,527,528
0,NaN,1682.870932,1610.899658,1569.892853,1519.308350,855.376494,789.789673,267.811067,236.602737,129.980806,...,4082.771162,4079.775391,3898.254940,3893.541992,3603.489933,3595.153320,3403.906424,3408.472656,3647.815922,3650.031006
1,NaN,1697.569767,1637.081421,1582.578522,1543.433838,862.772507,810.230957,279.417848,257.662537,121.348412,...,4063.495932,4060.885254,3892.599239,3890.902588,3642.685844,3637.665039,3400.070561,3403.722168,3645.860400,3645.820312
2,NaN,1691.078225,1620.775879,1609.190918,1556.537720,860.337983,805.921875,281.801626,255.386353,125.857288,...,4072.053807,4067.200195,3890.321200,3894.753662,3623.019971,3617.502197,3410.859397,3410.782471,3632.484807,3638.976074
3,NaN,1702.101509,1630.589355,1607.464267,1562.899658,849.476986,791.579468,281.374051,257.909973,128.275750,...,4078.613464,4074.968750,3896.750875,3897.742188,3641.734855,3631.630859,3411.606707,3411.334717,3641.445849,3640.393066
4,NaN,1716.043358,1638.569092,1635.874869,1576.715088,848.912162,797.978149,277.272182,250.664337,128.967759,...,4082.159589,4085.481445,3907.285632,3904.280273,3652.275745,3646.895508,3411.781533,3411.574707,3647.481306,3655.821533
5,NaN,1691.034632,1614.729370,1622.906392,1564.934814,859.117891,799.022888,264.920566,230.766510,131.593187,...,4055.162467,4062.105469,3870.212878,3859.594238,3628.046952,3619.007568,3405.849466,3404.781982,3628.166077,3625.409180
6,NaN,1673.823294,1603.500000,1600.222781,1546.626709,842.646347,789.558044,278.000390,253.288361,123.422571,...,4040.801396,4041.522461,3847.529533,3836.871094,3632.684269,3624.857910,3392.665340,3394.710449,3611.470886,3610.378906
7,NaN,1688.458262,1612.280518,1593.931163,1536.012451,852.184212,796.388245,270.668222,244.768661,128.202442,...,4075.132525,4078.982422,3893.439054,3887.946777,3636.124341,3625.458740,3406.173118,3408.410400,3642.739707,3642.506836
8,NaN,1686.885572,1613.916992,1625.520985,1569.752930,857.015054,798.990540,284.514449,265.606598,129.013541,...,4112.206922,4119.952148,3885.731283,3886.098633,3658.196586,3655.184326,3421.443595,3419.942139,3649.988307,3655.961914
9,NaN,1693.708359,1617.493408,1623.324618,1561.254395,847.924952,793.945312,267.773351,238.104431,134.179545,...,4093.776084,4098.359375,3885.594181,3886.109375,3637.486397,3631.974609,3413.216309,3421.449707,3644.558306,3650.301758


In [6]:
df = df.iloc[:,1::]

In [7]:
mean = df.iloc[:,1::2].to_numpy().astype(np.float32)
mean.shape

(269, 264)

In [8]:
hrf = gammaHRF(2.2, len_of_seconds = 591.5)
len(hrf.round(2))

269

In [12]:
269*2.2/16

36.987500000000004

In [9]:
esti = wiener_deconvolution(mean[:,0], hrf, 1e-3)

In [10]:
esti

array([1556.57547013, 1648.26787916, 1661.18950272, 1573.22969899,
       1597.85647285, 1633.33436111, 1617.13063717, 1594.28056693,
       1682.89107843, 1568.63738369, 1676.29934335, 1562.51727474,
       1670.49014769, 1620.97430595, 1606.48620267, 1604.47860835,
       1635.14352704, 1572.85044394, 1694.91407226, 1557.85023949,
       1667.01146498, 1601.65850641, 1502.48813719, 1699.91094553,
       1558.89807663, 1581.41339715, 1680.02002949, 1616.00096807,
       1608.21677748, 1565.60139143, 1661.11698039, 1556.81007349,
       1623.79796922, 1600.84419179, 1562.21965831, 1662.92204643,
       1567.57252326, 1620.978611  , 1596.91983605, 1622.37785725,
       1627.65103299, 1551.65889164, 1648.91434985, 1587.93944362,
       1583.25110836, 1630.04137223, 1535.9728305 , 1596.53175576,
       1673.81177582, 1511.56670361, 1669.96756171, 1600.45689172,
       1654.67276715, 1559.64437767, 1753.79575163, 1527.5426322 ,
       1739.70384752, 1451.32223113, 1809.43429847, 1444.19927

In [11]:
esti.shape

(269,)

In [200]:
from sklearn import preprocessing
scaler = preprocessing.StandardScaler().fit(esti2)
X_scaled = scaler.transform(esti2)

In [176]:
out2 = out.reshape(-1, 1)

In [177]:
scaler = preprocessing.StandardScaler().fit(out2)
out_scaled = scaler.transform(out2)

In [178]:
out_scaled

array([[-0.06346993],
       [ 2.04931286],
       [11.55344759],
       [10.2029363 ],
       [ 1.94039709],
       [-2.80226591],
       [-3.14574822],
       [-1.92565572],
       [-0.89093838],
       [-0.36228101],
       [-0.15579706],
       [-0.08868375],
       [-0.06969475],
       [-0.06488252],
       [-0.06376835],
       [-0.06352921],
       [-0.06348109],
       [-0.06347193],
       [-0.06347027],
       [-0.06346999],
       [-0.06346994],
       [-0.06346993],
       [-0.06346993],
       [-0.06346993],
       [-0.06346993],
       [-0.06346993],
       [-0.06346993],
       [-0.06346993],
       [-0.06346993],
       [-0.06346993],
       [-0.06346993],
       [-0.06346993],
       [-0.06346993],
       [-0.06346993],
       [-0.06346993],
       [-0.06346993],
       [-0.06346993],
       [-0.06346993],
       [-0.06346993],
       [-0.06346993],
       [-0.06346993],
       [-0.06346993],
       [-0.06346993],
       [-0.06346993],
       [-0.06346993],
       [-0

In [205]:
scaler = preprocessing.StandardScaler().fit(esti2)
X_scaled = scaler.transform(esti2)

In [14]:
from scipy.stats import gamma
import numpy as np
from math import ceil

def gammaHRF(TR, paras=None, len_of_seconds=32, onset_seconds=0):
    
    if paras is None:
        paras = [6,16,1,1,6]

    dt = TR/16
    u = np.array(range(0, ceil(len_of_seconds/dt))) - ceil(onset_seconds/dt)
    
    tmp1 = gamma.pdf(u, a = paras[0]/paras[2], scale = 1/(dt/paras[2])) 
    tmp2 = gamma.pdf(u, a = paras[1]/paras[3], scale = 1/(dt/paras[3]))/paras[4]       
    hrf = tmp1 - tmp2

    hrf = hrf[np.array(range(0, ceil(len_of_seconds/TR)))*16 + 1]

    hrf = hrf/sum(hrf)

    return hrf

from numpy.fft import fft, ifft, ifftshift

def wiener_deconvolution(signal, kernel, lambd):
    
    kernel = np.hstack((kernel, np.zeros(len(signal) - len(kernel)))) # zero pad the kernel to same length
    H = fft(kernel)
    deconvolved = np.real(ifft(fft(signal)*np.conj(H)/(H*np.conj(H) + lambd**2)))
    return deconvolved

import pandas as pd
import numpy as np

filename = "../3.application/NCANDA_S01010_run-01.feat__filtered_func.nii.gz.1D"
df = pd.read_csv(filename, sep='\t', header=None)
df = df.iloc[:,1::]

mean = df.iloc[:,1::2].to_numpy().astype(np.float32)
y = mean[:, 0]

hrf = gammaHRF(2.2, len_of_seconds = 37)

esti = wiener_deconvolution(mean[:,0], hrf, 1e-3)
esti

array([1556.57152363, 1648.27166243, 1661.18553251, 1573.23738477,
       1597.84863406, 1633.34229799, 1617.12570237, 1594.28471131,
       1682.88882551, 1568.63925662, 1676.30091724, 1562.5127581 ,
       1670.49742653, 1620.96968103, 1606.48649334, 1604.48043027,
       1635.14250843, 1572.84914711, 1694.91636634, 1557.84928341,
       1667.01192585, 1601.65759079, 1502.48928429, 1699.90794288,
       1558.90177729, 1581.40874057, 1680.02457794, 1615.99780915,
       1608.2181559 , 1565.60021254, 1661.11860758, 1556.80734489,
       1623.80139318, 1600.83969769, 1562.22301728, 1662.92015972,
       1567.5716437 , 1620.98281048, 1596.91485617, 1622.38087147,
       1627.65151797, 1551.65605424, 1648.91637582, 1587.93778432,
       1583.2533855 , 1630.03837648, 1535.97496918, 1596.53028738,
       1673.81213271, 1511.56799802, 1669.96502953, 1600.45956318,
       1654.6715809 , 1559.64470885, 1753.79709455, 1527.54056133,
       1739.70764124, 1451.31840269, 1809.43728079, 1444.19836